# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [122]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
filedir = '~/UCBERKDATA/REPO/UCBER201805DATA2-Class-Repository-DATA/02-Homework/04-Numpy-Pandas/Instructions/PyCitySchools/Resources/'
school_data_to_load = filedir+"schools_complete.csv"
student_data_to_load = filedir+"students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [123]:
total_number_school = len(school_data_complete['School ID'].unique().tolist())
total_number_student = len(school_data_complete['Student ID'].unique().tolist())
total_budget = school_data_complete.groupby('School ID')['budget'].mean().sum()
avg_math_score = school_data_complete['math_score'].mean()
avg_reading_score = school_data_complete['reading_score'].mean()
school_data_complete['Overall_average_score'] = 0.5 * (school_data_complete['reading_score'] + school_data_complete['math_score'])
#overall_passing_rate = school_data_complete['Overall_average_score'].mean()
student_passing_math_DF = school_data_complete.loc[school_data_complete['math_score']>=70]
student_passing_math_percentage = 100. * len(student_passing_math_DF)/total_number_student
student_passing_reading_DF = school_data_complete.loc[school_data_complete['reading_score']>=70]
student_passing_reading_percentage = 100. * len(student_passing_reading_DF)/total_number_student
overall_passing_rate = (student_passing_math_percentage + student_passing_reading_percentage)/2.

DSDF = pd.DataFrame({'Total Schools':[total_number_school], 'Total Students':[total_number_student],\
                     'Total Budget':[total_budget], 'Average Math Score':[avg_math_score],\
                     'Average Reading Score': [avg_reading_score], '% Passing Math': [student_passing_math_percentage],\
                     '% Passing Reading': [student_passing_reading_percentage], '% Overall passing rate': [overall_passing_rate]})
DSDF['Total Students'] = DSDF['Total Students'].map("{:,d}".format)
DSDF['Total Budget'] = DSDF['Total Budget'].map("${:,.2f}".format)
DSDF

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall passing rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [124]:
SSDF = school_data_complete.groupby(['school_name','type'])[['budget']].mean()
SSDF = SSDF.reset_index()
SSDF['Total Students'] = school_data_complete.groupby('school_name')['Student ID'].count().tolist()
SSDF = SSDF.rename(columns = {'school_name':'School Name', 'type':'School Type', 'budget':'Total School Budget'})
SSDF['Per Student Budget'] = SSDF['Total School Budget']/SSDF['Total Students']
SSDF['Average Math Score'] = school_data_complete.groupby('school_name')['math_score'].mean().tolist()
SSDF['Average Reading Score'] = school_data_complete.groupby('school_name')['reading_score'].mean().tolist()
school_data_complete['Passing math score'] = 1
school_data_complete['Passing reading score'] = 1
school_data_complete.loc[school_data_complete['math_score']<70,'Passing math score'] = np.nan
school_data_complete.loc[school_data_complete['reading_score']<70,'Passing reading score'] = np.nan
SSDF['% Passing Math'] = school_data_complete.groupby('school_name')['Passing math score'].count().tolist()/SSDF['Total Students']/0.01
SSDF['% Passing Reading'] = school_data_complete.groupby('school_name')['Passing reading score'].count().tolist()/SSDF['Total Students']/0.01
SSDF['% Overall Passing Rate'] = 0.5 * (SSDF['% Passing Math'] + SSDF['% Passing Reading'])
SSDF['Total School Budget'] = SSDF['Total School Budget'].map("${:,.2f}".format)
SSDF['Per Student Budget'] = SSDF['Per Student Budget'].map("${:,.2f}".format)
SSDF['Average Math Score'] = SSDF['Average Math Score'].map("{:.2f}".format)
SSDF['Average Reading Score'] = SSDF['Average Reading Score'].map("{:.2f}".format)
SSDF['% Passing Math'] = SSDF['% Passing Math'].map("{:.3f}".format)
SSDF['% Passing Reading'] = SSDF['% Passing Reading'].map("{:.3f}".format)
SSDF['% Overall Passing Rate'] = SSDF['% Overall Passing Rate'].map("{:.3f}".format)
SSDF = SSDF.set_index('School Name')
SSDF

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,"$3,124,928.00",4976,$628.00,77.05,81.03,66.680,81.933,74.307
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.06,83.98,94.133,97.040,95.587
Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.71,81.16,65.988,80.739,73.364
Ford High School,District,"$1,763,916.00",2739,$644.00,77.10,80.75,68.310,79.299,73.804
Griffin High School,Charter,"$917,500.00",1468,$625.00,83.35,83.82,93.392,97.139,95.266
Hernandez High School,District,"$3,022,020.00",4635,$652.00,77.29,80.93,66.753,80.863,73.808
Holden High School,Charter,"$248,087.00",427,$581.00,83.80,83.81,92.506,96.253,94.379
Huang High School,District,"$1,910,635.00",2917,$655.00,76.63,81.18,65.684,81.316,73.500
Johnson High School,District,"$3,094,650.00",4761,$650.00,77.07,80.97,66.058,81.222,73.640


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [125]:
SSDF2 = SSDF.sort_values(by=['% Overall Passing Rate'],ascending=False)
SSDF2.head(5)

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,83.06,83.98,94.133,97.040,95.587
Thomas High School,Charter,"$1,043,130.00",1635,$638.00,83.42,83.85,93.272,97.309,95.291
Pena High School,Charter,"$585,858.00",962,$609.00,83.84,84.04,94.595,95.946,95.270
Griffin High School,Charter,"$917,500.00",1468,$625.00,83.35,83.82,93.392,97.139,95.266
Wilson High School,Charter,"$1,319,574.00",2283,$578.00,83.27,83.99,93.868,96.540,95.204


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [126]:
SSDF3 = SSDF.sort_values(by=['% Overall Passing Rate'])
SSDF3.head(5)

,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"$2,547,363.00",3999,$637.00,76.84,80.74,66.367,80.220,73.293
Figueroa High School,District,"$1,884,411.00",2949,$639.00,76.71,81.16,65.988,80.739,73.364
Huang High School,District,"$1,910,635.00",2917,$655.00,76.63,81.18,65.684,81.316,73.500
Johnson High School,District,"$3,094,650.00",4761,$650.00,77.07,80.97,66.058,81.222,73.640
Ford High School,District,"$1,763,916.00",2739,$644.00,77.10,80.75,68.310,79.299,73.804


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [127]:
MSGDF = school_data_complete.groupby(['school_name','grade'])[['math_score']].mean()
MSGDF = MSGDF.reset_index()
p_MSGDF = MSGDF.pivot(index='school_name',columns='grade',values='math_score')
cols = p_MSGDF.columns.tolist()
newcols = cols[-1:] + cols[:-1]
p_MSGDF = p_MSGDF[newcols]
p_MSGDF

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [128]:
RSGDF = school_data_complete.groupby(['school_name','grade'])[['reading_score']].mean()
RSGDF = RSGDF.reset_index()
p_RSGDF = RSGDF.pivot(index='school_name',columns='grade',values='reading_score')
cols = p_RSGDF.columns.tolist()
newcols = cols[-1:] + cols[:-1]
p_RSGDF = p_RSGDF[newcols]
p_RSGDF

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [129]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [130]:
SSDF5 = school_data_complete.groupby(['school_name'])[['budget']].mean()
SSDF5['Total Students'] = school_data_complete.groupby('school_name')['Student ID'].count().tolist()
SSDF5 = SSDF5.rename(columns = {'school_name':'School Name', 'type':'School Type', 'budget':'Total School Budget'})
SSDF5['Per Student Budget'] = SSDF5['Total School Budget']/SSDF5['Total Students']
SSDF5['Average Math Score'] = school_data_complete.groupby('school_name')['math_score'].mean().tolist()
SSDF5['Average Reading Score'] = school_data_complete.groupby('school_name')['reading_score'].mean().tolist()
SSDF5['% Passing Math'] = pd.DataFrame(school_data_complete.groupby('school_name')['Passing math score'].count().tolist()/SSDF['Total Students']/0.01)
SSDF5['% Passing Reading'] = school_data_complete.groupby('school_name')['Passing reading score'].count().tolist()/SSDF['Total Students']/0.01
SSDF5['% Overall Passing Rate'] = 0.5* (SSDF5['% Passing Math'] + SSDF5['% Passing Reading'])
SSBDF = SSDF5.groupby(pd.cut(SSDF5['Per Student Budget'],spending_bins,labels=group_names))[['Average Math Score']].mean()
SSBDF['Average Reading Score'] = SSDF5.groupby(pd.cut(SSDF5['Per Student Budget'],spending_bins,labels=group_names))['Average Reading Score'].mean()
SSBDF['% Passing Math'] = SSDF5.groupby(pd.cut(SSDF5['Per Student Budget'],spending_bins,labels=group_names))['% Passing Math'].mean()
SSBDF['% Passing Reading'] = SSDF5.groupby(pd.cut(SSDF5['Per Student Budget'],spending_bins,labels=group_names))['% Passing Reading'].mean()
SSBDF['% Overall Passing Rate'] = SSDF5.groupby(pd.cut(SSDF5['Per Student Budget'],spending_bins,labels=group_names))['% Overall Passing Rate'].mean()
SSBDF

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [131]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [132]:
SSBDF2 = SSDF5.groupby(pd.cut(SSDF5['Total Students'],size_bins,labels=group_names))[['Average Math Score']].mean()
SSBDF2['Average Reading Score'] = SSDF5.groupby(pd.cut(SSDF5['Total Students'],size_bins,labels=group_names))['Average Reading Score'].mean()
SSBDF2['% Passing Math'] = SSDF5.groupby(pd.cut(SSDF5['Total Students'],size_bins,labels=group_names))['% Passing Math'].mean()
SSBDF2['% Passing Reading'] = SSDF5.groupby(pd.cut(SSDF5['Total Students'],size_bins,labels=group_names))['% Passing Reading'].mean()
SSBDF2['% Overall Passing Rate'] = SSDF5.groupby(pd.cut(SSDF5['Total Students'],size_bins,labels=group_names))['% Overall Passing Rate'].mean()
SSBDF2

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [133]:
SSDF5['School Type'] = SSDF2['School Type']
SSBDF3 = SSDF5.groupby('School Type')[['Average Math Score']].mean()
SSBDF3['Average Reading Score'] = SSDF5.groupby('School Type')[['Average Reading Score']].mean()
SSBDF3['% Passing Math'] = SSDF5.groupby('School Type')[['% Passing Math']].mean()
SSBDF3['% Passing Reading'] = SSDF5.groupby('School Type')[['% Passing Reading']].mean()
SSBDF3['% Overall Passing Rate'] = SSDF5.groupby('School Type')[['% Overall Passing Rate']].mean()
SSBDF3

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
